In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

from google.colab import drive
drive.mount("/content/drive")

# Define directories for images and masks
image_dir = "/content/drive/MyDrive/RcNN/Images"
mask_dir = "/content/drive/MyDrive/RcNN/Masks"
import os

# List the contents of the image directory
image_files = os.listdir(image_dir)
print("Image files in directory:")
print(image_files)

# List the contents of the mask directory
mask_files = os.listdir(mask_dir)
print("Mask files in directory:")
print(mask_files)


from PIL import Image

def load_data(image_dir, mask_dir, target_size=(128, 128,3)):
    print("Loading data...")
    images = []
    masks = []
    count = 0
    for image_name in os.listdir(image_dir):
        count += 1
        if count < 200 and len(image_name) <= 19:  # Limiting to the first 1000 images for demonstration
            image_path = os.path.join(image_dir, image_name)
            mask_path = os.path.join(mask_dir, image_name.split(".")[0] + ".jpg")  # Assuming mask files follow a naming convention

            print(count)


            # Load and resize image
            image = Image.open(image_path)
            image = image.resize(target_size)
            image = np.array(image)

            # Load and resize mask
            mask = Image.open(mask_path)
            mask = mask.resize(target_size)
            mask = np.array(mask)

            if image is not None and mask is not None:
                images.append(image)
                masks.append(mask)
    return np.array(images), np.array(masks)


# Load data
images, masks = load_data(image_dir, mask_dir)

print("Number of images:", len(images))
print("Number of masks:", len(masks))
# Split data into train and test sets
train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

# Define RCNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Single neuron for binary classification
])


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Train the model
model.fit(train_images, train_masks, epochs=10, batch_size=32, validation_data=(test_images, test_masks))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_masks)
print("Test accuracy:", test_acc)

# Display a sample prediction
sample_index = 0
sample_image = test_images[sample_index]
sample_mask = test_masks[sample_index]

plt.subplot(1, 2, 1)
plt.imshow(sample_image)
plt.title("Sample Image")

plt.subplot(1, 2, 2)
plt.imshow(sample_mask, cmap="gray")
plt.title("Sample Mask")

plt.show()


ValueError: mount failed